Game defined as :

    *State: 
        - Grid size
        - Positions of living cells
    *Rules: eg
        - A cell survives if it has 2 or 3 neighbours -> dies if 1 or 0 neighbours 
        - Dies if it has more than 3 neighbours.
        - If a dead cell has 3 neighbours it becomes alive.
    *Stop test:
        - If current state = previous state 
        

In [1]:
%matplotlib inline
import IPython.display
from io import BytesIO as bio
import PIL.Image
import numpy as np
from abc import ABC, abstractmethod
from copy import copy
import matplotlib.pyplot as plt
import matplotlib
import time
# matplotlib.use('GTK3Agg')

In [2]:
class State(ABC):
    @abstractmethod
    def copy(self):
        pass

    @abstractmethod
    def apply_rules(self, rules, max_size):
        pass

    @abstractmethod
    def equals(self, other):
        pass

    @abstractmethod
    def get_neighbours(self, elem, max_size):
        pass


class DenseNumpyState(State):
    def __init__(self, grid):
        self.grid = grid

    def copy(self):
        return DenseNumpyState(np.copy(self.grid))

    def equals(self, other):
        if other is None:
            return False
        return np.array_equal(self.grid, other.grid)

    def apply_rules(self, rules, max_size,):
        self.grid = rules.apply_rules(self.grid, max_size, self.get_neighbours)
        return self

    def get_neighbours(self, elem, max_size):
        l = []
        if elem[0]-1 >= 0:
            l.append((elem[0]-1, elem[1]))
        if elem[0]-1 >= 0 and elem[1]-1 >= 0:
            l.append((elem[0]-1, elem[1]-1))
        if elem[0]-1 >= 0 and elem[1]+1 < max_size:
            l.append((elem[0]-1, elem[1]+1))
        if elem[1]-1 >= 0:
            l.append((elem[0], elem[1]-1))
        if elem[1]-1 >= 0 and elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]-1))
        if elem[1]+1 < max_size:
            l.append((elem[0], elem[1]+1))
        if elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]))
        if elem[1]+1 < max_size and elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]+1))
        return l


class SparseSetState(State):
    def __init__(self, grid):
        self.grid = grid

    def copy(self):
        return SparseSetState(copy(self.grid))

    def get_neighbours(self, elem, max_size):
        # Returns the neighbours of a live cell if they lie within the bounds of the grid specified by max_size
        l = []
        if elem[0]-1 >= 0:
            l.append((elem[0]-1, elem[1]))
        if elem[0]-1 >= 0 and elem[1]-1 >= 0:
            l.append((elem[0]-1, elem[1]-1))
        if elem[0]-1 >= 0 and elem[1]+1 < max_size:
            l.append((elem[0]-1, elem[1]+1))
        if elem[1]-1 >= 0:
            l.append((elem[0], elem[1]-1))
        if elem[1]-1 >= 0 and elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]-1))
        if elem[1]+1 < max_size:
            l.append((elem[0], elem[1]+1))
        if elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]))
        if elem[1]+1 < max_size and elem[0]+1 < max_size:
            l.append((elem[0]+1, elem[1]+1))
        return l

    def equals(self, other):
        if other is None:
            return False
        return self.grid == other.grid

    def apply_rules(self, rules, max_size):
        # Calls the actual rules and provides them with the grid and the neighbour function
        self.grid = rules.apply_rules(self.grid, max_size, self.get_neighbours)
        return self

# %%pixie_debugger
class Rule(ABC):
    @abstractmethod
    def apply_rules(self, grid, max_size, get_neighbours):
        pass


class DenseNumpyRules(Rule):
    def apply_rules(self, grid, max_size, get_neighbours):
        #copied_state = state.copy()
        #grid = state.grid

        grid_ret = copy(grid)
        for i in range(grid.shape[0]):
            for j in range(grid.shape[1]):
                nb = get_neighbours((i, j), max_size)
                counter = 0
                for n in nb:
                    if grid[n] == True:
                        counter += 1
                if (counter < 2 or counter > 3):
                    grid_ret[i][j] = False
                if (counter == 3):
                    grid_ret[i][j] = True
        return grid_ret


class SparseSetRules(Rule):
    def apply_rules(self, grid, max_size, get_neighbours):
        #grid = state.grid
        counter = {}
        for elem in grid:
            if elem not in counter:
                counter[elem] = 0
            nb = get_neighbours(elem, max_size)
            for n in nb:
                if n not in counter:
                    counter[n] = 1
                else:
                    counter[n] += 1
        for c in counter:
            if (counter[c] < 2 or counter[c] > 3):
                grid.discard(c)
            if counter[c] == 3:
                grid.add(c)
        return grid

class Game:
    def __init__(self, initial_state, rules,max_size):
        self.initial_state = initial_state
        self.rules = rules
        self.max_size = max_size
    def run_game(self, it):
        state = self.initial_state
        previous_state = None
        progression = []
        i = 0
        while (not state.equals(previous_state) and i < it):
            i += 1
            previous_state = state.copy()
            progression.append(previous_state.grid)
            state = state.apply_rules(self.rules,self.max_size)
        progression.append(state.grid)
        return progression

def random_grid(size):
    randoms = np.random.randint(2, size=size*size)
    board = set()
    for index, elem in enumerate(randoms):
        if elem == 1:
            x = index%size
            y = int(index/size)
            board.add((x, y))
    return board

In [3]:
#transform array to a gif and save to a file
def save_gif(array,file_name):
    array = np.uint8(np.clip(array,0,1)*255.0)
    frames = []
    for frame in range(array.shape[0]):
        img = PIL.Image.fromarray(array[frame])
        img = img.resize((500, 500))
        frames.append(img)
    img.save(file_name, save_all=True, duration=33.33, append_images=frames, loop=0,size=(500,500))

In [4]:
#transform array to a gif and save to a file
def save_img(array, file_name):
    array = np.uint8(np.clip(array,0,1)*255.0)
    print(array.shape[0])
    img = PIL.Image.fromarray(array[array.shape[0] - 1 ])
    img = img.resize((500, 500))
    img.save(file_name, size=(500,500))

In [21]:
MAX_ITER = 1000
MAX_SIZE = 20
GIF_FOLDER = "gif/"
IMG_FOLDER = "img/"

def generate(number):
    number = str(number)
    init = np.zeros((MAX_SIZE, MAX_SIZE), dtype=bool)
    board = random_grid(MAX_SIZE)

    rules2 = SparseSetRules()
    game = Game(SparseSetState(board), rules2,MAX_SIZE)
    t = time.time()
    rw = game.run_game(MAX_ITER)
    print("Iterations passed: {}, time: {}".format(len(rw), time.time()-t))

    #transform sparse representation to array for plotting
    res = np.zeros((len(rw), MAX_SIZE, MAX_SIZE), dtype=bool)
    for l in range(0,len(rw)):
        for key in rw[l]:
            res[l,key[0], key[1]] = True 

    save_img(res,IMG_FOLDER + number + ".png")
    save_gif(res,GIF_FOLDER + number + ".gif")

In [26]:
for i in range(102,1001):
    generate(i)

Iterations passed: 1001, time: 0.12668514251708984
1001
Iterations passed: 83, time: 0.023953914642333984
83
Iterations passed: 1001, time: 0.07863521575927734
1001
Iterations passed: 65, time: 0.016825437545776367
65
Iterations passed: 1001, time: 0.09569525718688965
1001
Iterations passed: 151, time: 0.029912233352661133
151
Iterations passed: 53, time: 0.01714015007019043
53
Iterations passed: 179, time: 0.03589057922363281
179
Iterations passed: 143, time: 0.035062551498413086
143
Iterations passed: 1001, time: 0.13235902786254883
1001
Iterations passed: 1001, time: 0.062079668045043945
1001
Iterations passed: 58, time: 0.019895553588867188
58
Iterations passed: 148, time: 0.033933401107788086
148
Iterations passed: 136, time: 0.03913593292236328
136
Iterations passed: 225, time: 0.06991958618164062
225
Iterations passed: 1001, time: 0.14482951164245605
1001
Iterations passed: 49, time: 0.011309385299682617
49
Iterations passed: 63, time: 0.016433000564575195
63
Iterations passed: 

Iterations passed: 102, time: 0.04206228256225586
102
Iterations passed: 59, time: 0.013431072235107422
59
Iterations passed: 113, time: 0.035530805587768555
113
Iterations passed: 1001, time: 0.17875242233276367
1001
Iterations passed: 106, time: 0.02605581283569336
106
Iterations passed: 1001, time: 0.0982048511505127
1001
Iterations passed: 1001, time: 0.0562283992767334
1001
Iterations passed: 1001, time: 0.24233078956604004
1001
Iterations passed: 428, time: 0.24491524696350098
428
Iterations passed: 59, time: 0.03052210807800293
59
Iterations passed: 192, time: 0.060575008392333984
192
Iterations passed: 213, time: 0.04984641075134277
213
Iterations passed: 67, time: 0.009611368179321289
67
Iterations passed: 1001, time: 0.1267540454864502
1001
Iterations passed: 1001, time: 0.15189790725708008
1001
Iterations passed: 1001, time: 0.2960789203643799
1001
Iterations passed: 1001, time: 0.1836717128753662
1001
Iterations passed: 1001, time: 0.15403008460998535
1001
Iterations passed

Iterations passed: 1001, time: 0.08744025230407715
1001
Iterations passed: 32, time: 0.010320901870727539
32
Iterations passed: 1001, time: 0.19350194931030273
1001
Iterations passed: 176, time: 0.04864501953125
176
Iterations passed: 1001, time: 0.09531855583190918
1001
Iterations passed: 146, time: 0.0280454158782959
146
Iterations passed: 100, time: 0.024103403091430664
100
Iterations passed: 1001, time: 0.13531112670898438
1001
Iterations passed: 1001, time: 0.11762714385986328
1001
Iterations passed: 1001, time: 0.08656048774719238
1001
Iterations passed: 1001, time: 0.09208893775939941
1001
Iterations passed: 252, time: 0.05666327476501465
252
Iterations passed: 148, time: 0.038413047790527344
148
Iterations passed: 130, time: 0.04027390480041504
130
Iterations passed: 1001, time: 0.05882740020751953
1001
Iterations passed: 53, time: 0.010278463363647461
53
Iterations passed: 134, time: 0.023576974868774414
134
Iterations passed: 102, time: 0.024079322814941406
102
Iterations pas

Iterations passed: 1001, time: 0.17680597305297852
1001
Iterations passed: 1001, time: 0.1020817756652832
1001
Iterations passed: 65, time: 0.02461981773376465
65
Iterations passed: 64, time: 0.022372007369995117
64
Iterations passed: 56, time: 0.013252973556518555
56
Iterations passed: 130, time: 0.032450199127197266
130
Iterations passed: 78, time: 0.012215375900268555
78
Iterations passed: 43, time: 0.013109207153320312
43
Iterations passed: 1001, time: 0.14623689651489258
1001
Iterations passed: 138, time: 0.03615164756774902
138
Iterations passed: 1001, time: 0.10733652114868164
1001
Iterations passed: 1001, time: 0.16912078857421875
1001
Iterations passed: 111, time: 0.029044151306152344
111
Iterations passed: 102, time: 0.02813577651977539
102
Iterations passed: 179, time: 0.04423689842224121
179
Iterations passed: 200, time: 0.050533294677734375
200
Iterations passed: 85, time: 0.026807546615600586
85
Iterations passed: 1001, time: 0.07678389549255371
1001
Iterations passed: 10

Iterations passed: 1001, time: 0.15692996978759766
1001
Iterations passed: 1001, time: 0.12362194061279297
1001
Iterations passed: 1001, time: 0.0865790843963623
1001
Iterations passed: 65, time: 0.01942729949951172
65
Iterations passed: 1001, time: 0.10626864433288574
1001
Iterations passed: 30, time: 0.008788824081420898
30
Iterations passed: 252, time: 0.055391550064086914
252
Iterations passed: 21, time: 0.005586862564086914
21
Iterations passed: 230, time: 0.04387497901916504
230
Iterations passed: 1001, time: 0.08377599716186523
1001
Iterations passed: 1001, time: 0.08792257308959961
1001
Iterations passed: 1001, time: 0.07809114456176758
1001
Iterations passed: 130, time: 0.035666704177856445
130
Iterations passed: 1001, time: 0.09055233001708984
1001
Iterations passed: 1001, time: 0.15184330940246582
1001
Iterations passed: 1001, time: 0.09411978721618652
1001
Iterations passed: 30, time: 0.006864309310913086
30
Iterations passed: 1001, time: 0.0806267261505127
1001
Iterations 

Iterations passed: 1001, time: 0.07649111747741699
1001
Iterations passed: 1001, time: 0.17744183540344238
1001
Iterations passed: 130, time: 0.025847911834716797
130
Iterations passed: 366, time: 0.09053611755371094
366
Iterations passed: 1001, time: 0.06334757804870605
1001
Iterations passed: 1001, time: 0.10136532783508301
1001
Iterations passed: 1001, time: 0.14464998245239258
1001
Iterations passed: 102, time: 0.03057074546813965
102
Iterations passed: 163, time: 0.0273592472076416
163
Iterations passed: 46, time: 0.01107335090637207
46
Iterations passed: 61, time: 0.011228561401367188
61
Iterations passed: 122, time: 0.029386043548583984
122
Iterations passed: 199, time: 0.09256482124328613
199
Iterations passed: 73, time: 0.025450944900512695
73
Iterations passed: 105, time: 0.028900146484375
105
Iterations passed: 61, time: 0.018178701400756836
61
Iterations passed: 441, time: 0.09737849235534668
441
Iterations passed: 94, time: 0.01798105239868164
94
Iterations passed: 1001, t